In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [2]:
base = pd.read_csv('test.csv')

In [3]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [4]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape, x.shape

((3000,), (3000, 1000))

In [5]:
y = y.values.reshape(-1,1)
y.shape

(3000, 1)

In [6]:
y = onehot.fit_transform(y).toarray()

In [7]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.4)

In [8]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((3000, 1000), (1200, 1000), (3000, 3), (1200, 3))

In [9]:
neuronios_entrada = x.shape[1]
neuronios_entrada

1000

In [10]:
neuronios_oculta1 = int((neuronios_entrada + y.shape[1])/2)
neuronios_oculta1

501

In [11]:
neuronios_saida = y.shape[1]
neuronios_saida

3

In [12]:
# 100 -> 51 -> 3

In [13]:
w = {'oculta1': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta1])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta1, neuronios_saida]))}

In [14]:
b = {'oculta1': tf.Variable(tf.random_normal([neuronios_oculta1])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}

In [15]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [16]:
def mlp(x, w, bias):
    camada_oculta1 = tf.nn.relu(tf.add(tf.matmul(x, w['oculta1']), b['oculta1']))
    camada_saida = tf.add(tf.matmul(camada_oculta1, w['saida']), b['saida'])
    return camada_saida

In [17]:
modelo = mlp(xph, w, b)

In [18]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(erro)

In [19]:
batch_size = 50
batch_total = int(len(x_treinamento) / batch_size)
batch_total, x_treinamento.shape

(36, (1800, 1000))

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(500):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 30 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])
    
    print('fim do treinamento')
    
    previsoes = tf.nn.softmax(mlp(xph, w_final, b_final))
    previsoes_corretas = tf.equal(tf.argmax(previsoes, 1), tf.argmax(yph, 1))
    taxa_acerto = tf.reduce_mean(tf.cast(previsoes_corretas, tf.float32))
    print(sess.run(taxa_acerto, feed_dict = {xph: x_teste, yph: y_teste}))

Época: 1 erro: 1496.766369289822
Época: 31 erro: 1.5649174448942682
Época: 61 erro: 0.06481149561099205
Época: 91 erro: 0.00012114110423903458
Época: 121 erro: 5.4020973733143793e-05
Época: 151 erro: 2.9808139087224763e-05
Época: 181 erro: 1.7433926020580008e-05
Época: 211 erro: 1.0433088312833402e-05
Época: 241 erro: 6.294189499205732e-06
Época: 271 erro: 3.7831132463978567e-06
Época: 301 erro: 2.2547304500747997e-06
Época: 331 erro: 1.3349724456003363e-06
Época: 361 erro: 7.885559831777299e-07
Época: 391 erro: 4.6729577698714473e-07
Época: 421 erro: 2.7623282545303283e-07
Época: 451 erro: 1.6417713234585637e-07
Época: 481 erro: 9.755274678181614e-08
fim do treinamento
0.98583335
